In [1]:
#!/usr/bin/env python3
import os
import sys

# BEGIN THREAD SETTINGS this sets the number of threads used by numpy in the program
# (should be set to 1 to avoid implicit parallelism)
implicit_num_threads = 1
os.environ["OMP_NUM_THREADS"] = str(implicit_num_threads)
os.environ["MKL_NUM_THREADS"] = str(implicit_num_threads)
os.environ["OPENBLAS_NUM_THREADS"] = str(implicit_num_threads)
# END THREAD SETTINGS

import numpy as np
import torch
from numpy import random
import matplotlib
import pickle
matplotlib.use('agg')
from matplotlib import pyplot as plt
import threading
import time
import pandas
from collections import deque

from tqdm import tqdm
from google.colab import files, drive

In [2]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
path = "/content/gdrive/MyDrive/data_daily.csv"
# reading the CSV file
csvFile = pandas.read_csv(path)
 
# displaying the contents of the CSV file
print(csvFile)

         # Date  Receipt_Count
0    2021-01-01        7564766
1    2021-01-02        7455524
2    2021-01-03        7095414
3    2021-01-04        7666163
4    2021-01-05        7771289
..          ...            ...
360  2021-12-27       10350408
361  2021-12-28       10219445
362  2021-12-29       10313337
363  2021-12-30       10310644
364  2021-12-31       10211187

[365 rows x 2 columns]


In [4]:
def train_test_split(Xs, Ys, test_size, random_state):
    random.seed(random_state)
    random.shuffle(Xs)
    random.shuffle(Ys)
    test_set_size = int(test_size * len(Ys))
    Xs_tr, Xs_va, Ys_tr, Ys_va = Xs[test_set_size:], Xs[:test_set_size], Ys[test_set_size:], Ys[:test_set_size]
    return Xs_tr, Xs_va, Ys_tr, Ys_va

In [5]:
# setting the days ahead range for predicting the approximate number of the scanned receipts for a future day
day_range = 30 # using the data from day 0 to day 29 to predict day 30

# constructing features and target variables
Receipt_Count = csvFile["Receipt_Count"].array.to_numpy()
Xs = [[Receipt_Count[j] for j in range(i, i + day_range)] for i in range(335)]
Ys = Receipt_Count[day_range:]
assert len(Xs) == len(Ys)
# perform train-validation (0.8 vs 0.2) split
Xs_tr, Xs_va, Ys_tr, Ys_va = train_test_split(Xs, 
                                              Ys, 
                                              test_size = 0.2, 
                                              random_state = 123)

In [6]:
# normalization of input data
mean = np.mean(Receipt_Count)
std = np.std(Receipt_Count)
Xs_tr = np.array(Xs_tr, dtype=float).reshape(day_range, -1)
Xs_va = np.array(Xs_va, dtype=float).reshape(day_range, -1)
Ys_tr = np.array(Ys_tr, dtype=float).reshape(1, -1)
Ys_va = np.array(Ys_va, dtype=float).reshape(1, -1)
Xs_tr = (Xs_tr - mean) / std
Ys_tr = (Ys_tr - mean) / std
Xs_tr = (Xs_va - mean) / std
Ys_tr = (Ys_va - mean) / std
receipts_dataset = (Xs_tr, Xs_va, Ys_tr, Ys_va)
print(Xs_tr.shape, Xs_va.shape, Ys_tr.shape, Ys_va.shape)

(30, 67) (30, 67) (1, 67) (1, 67)


In [7]:
# weight matrix initialization
h = 5
W10 = np.zeros((h, len(Xs_tr)))
W20 = np.zeros((len(Ys_tr), h))
print(W10.shape, W20.shape)

(5, 30) (1, 5)


In [8]:
### a function to create a unique increasing ID
### note that this is just a quick-and-easy way to create a global order
### it's not the only way to do it
global_order_counter = 0
def get_next_order():
    global global_order_counter
    rv = global_order_counter
    global_order_counter = global_order_counter + 1
    return rv

In [9]:
### a helper function to convert constants into BackproppableArray objects
def to_ba(x):
    if isinstance(x, BackproppableArray):
        return x
    elif isinstance(x, np.ndarray):
        return BackproppableArray(x)
    elif isinstance(x, float):
        return BackproppableArray(np.array(x))
    elif isinstance(x, int):
        return BackproppableArray(np.array(float(x)))
    else:
        raise Exception("could not convert {} to BackproppableArray".format(x))

In [10]:
### a class for an array that can be "packpropped-through"
class BackproppableArray(object):
    # np_array     numpy array that stores the data for this object
    def __init__(self, np_array, dependencies=[]):
        super().__init__()
        self.data = np_array

        # grad holds the gradient, an array of the same shape as data
        # before backprop, grad is None
        # during backprop before grad_fn is called, grad holds the partially accumulated gradient
        # after backprop, grad holds the gradient of the loss (the thing we call backward on)
        #     with respect to this array
        # if you want to use the same array object to call backward twice, you need to re-initialize
        #     grad to zero first
        self.grad = np.zeros(np_array.shape, dtype="float64")

        # an counter that increments monotonically over the course of the application
        # we know that arrays with higher order must depend only on arrays with lower order
        # we can use this to order the arrays for backpropagation
        self.order = get_next_order()

        # a list of other BackproppableArray objects on which this array directly depends
        # we'll use this later to decide which BackproppableArray objects need to participate in the backward pass
        self.dependencies = dependencies

    # represents me as a string
    def __repr__(self):
        return "({}, type={})".format(self.data, type(self).__name__)

    # returns a list containing this array and ALL the dependencies of this array, not just
    #    the direct dependencies listed in self.dependencies
    # that is, this list should include this array, the arrays in self.dependencies,
    #     plus all the arrays those arrays depend on, plus all the arrays THOSE arrays depend on, et cetera
    # the returned list must only include each dependency ONCE
    def all_dependencies(self):
        # TODO: (1.1) implement some sort of search to get all the dependencies
        res = [self]
        if not self.dependencies: return res
        q = deque()
        for i in range(len(self.dependencies)):
            q.append(self.dependencies[i])
        while q:
            for _ in range(len(q)):
                node = q.popleft()
                if node not in res:
                    node.grad = np.zeros(node.data.shape, dtype="float64")
                    res.append(node)
                for j in range(len(node.dependencies)):
                    q.append(node.dependencies[j])
        return res
            

    # compute gradients of this array with respect to everything it depends on
    def backward(self):
        # can only take the gradient of a scalar
        #assert(self.data.size == 1)

        # depth-first search to find all dependencies of this array
        all_my_dependencies = self.all_dependencies()
        # TODO: (1.2) implement the backward pass to compute the gradients
        #   this should do the following
        #   (1) sort the found dependencies so that the ones computed last go FIRST
        all_my_dependencies = sorted(all_my_dependencies, key = lambda x: x.order, reverse = True)
        #   (2) initialize and zero out all the gradient accumulators (.grad) for all the dependencies
        for dependency in all_my_dependencies:
            if len(dependency.data.shape)==0:
                dependency.grad = np.array(0.0)
            else:
                dependency = np.zeros(dependency.data.shape, dtype="float64")
        #   (3) set the gradient accumulator of this array to 1, as an initial condition
        #           since the gradient of a number with respect to itself is 1
        if len(self.data.shape)==0:
            self.grad = np.array(1.0)
        else:
            self.grad = np.ones(self.data.shape, dtype="float64")
        #   (4) call the backward function for all the dependencies in the sorted reverse order
        for dependency in all_my_dependencies:
            dependency.grad_fn()

    # function that is called to process a single step of backprop for this array
    # when called, it must be the case that self.grad contains the gradient of the loss (the
    #     thing we are differentating) with respect to this array
    # this function should update the .grad field of its dependencies
    #
    # this should just say "pass" for the parent class
    #
    # child classes override this
    def grad_fn(self):
        pass

    # operator overloading
    def __add__(self, other):
        return BA_Add(self, to_ba(other))
    def __sub__(self, other):
        return BA_Sub(self, to_ba(other))
    def __mul__(self, other):
        return BA_Mul(self, to_ba(other))
    def __truediv__(self, other):
        return BA_Div(self, to_ba(other))

    def __radd__(self, other):
        return BA_Add(to_ba(other), self)
    def __rsub__(self, other):
        return BA_Sub(to_ba(other), self)
    def __rmul__(self, other):
        return BA_Mul(to_ba(other), self)
    def __rtruediv__(self, other):
        return BA_Div(to_ba(other), self)

    # TODO (2.2) Add operator overloading for matrix multiplication
    def __matmul__(self, other):
        return BA_MatMul(self, to_ba(other))
    def __rmatmul__(self, other):
        return BA_MatMul(to_ba(other), self)
    
    def sum(self, axis=None, keepdims=True):
        return BA_Sum(self, axis)

    def reshape(self, shape):
        return BA_Reshape(self, shape)

    def transpose(self, axes = None):
        if axes is None:
            axes = range(self.data.ndim)[::-1]
        return BA_Transpose(self, axes)

In [11]:
# a class for an array that's the result of an addition operation
class BA_Add(BackproppableArray):
    # x + y
    def __init__(self, x, y):
        super().__init__(x.data + y.data, [x,y])
        self.x = x
        self.y = y

    def grad_fn(self):
        # TODO: (2.3) improve grad fn for Add
        if len(self.x.data.shape)==0:
            self.x.grad = self.x.grad+self.grad
        else:
        
            if len(self.x.data) != 1:
                self.x.grad = self.x.grad+self.grad
            else:
                self.x.grad = self.x.grad+np.sum(self.grad)
        if len(self.y.data.shape)==0:
            self.y.grad = self.y.grad+self.grad
        else:
            if len(self.y.data) != 1:
                self.y.grad = self.y.grad+self.grad
            else:
                self.y.grad = self.y.grad+np.sum(self.grad)

In [12]:
# a class for an array that's the result of a subtraction operation
class BA_Sub(BackproppableArray):
    # x + y
    def __init__(self, x, y):
        super().__init__(x.data - y.data, [x,y])
        self.x = x
        self.y = y

    def grad_fn(self):
        # TODO: (1.3, 2.3) implement grad fn for Sub
        if len(self.x.data.shape)==0:
            self.x.grad = self.x.grad+self.grad
        else:
        
            if len(self.x.data) != 1:
                self.x.grad = self.x.grad++self.grad
            else:
                self.x.grad = self.x.grad++np.sum(self.grad)
        if len(self.y.data.shape)==0:
            self.y.grad = self.y.grad-self.grad
        else:
            if len(self.y.data) != 1:
                self.y.grad = self.y.grad+-self.grad
            else:
                self.y.grad = self.y.grad-np.sum(self.grad)

In [13]:
# a class for an array that's the result of a multiplication operation
class BA_Mul(BackproppableArray):
    # x * y
    def __init__(self, x, y):
        super().__init__(x.data * y.data, [x,y])
        self.x = x
        self.y = y

    def grad_fn(self):
        # TODO: (1.3, 2.3) implement grad fn for Mul
        if len(self.x.data.shape)==0:
            self.x.grad = self.x.grad+self.y.data*self.grad
        else:
            if len(self.x.data) != 1:
                self.x.grad = self.x.grad+self.y.data*self.grad
            else:
                self.x.grad = self.x.grad+np.sum(self.grad*self.y.data)
        if len(self.y.data.shape)==0:
            self.y.grad = self.y.grad+self.x.data*self.grad
        else:
            if len(self.y.data) != 1:
                self.y.grad = self.y.grad+self.x.data*self.grad
            else:
                self.y.grad = self.y.grad+np.sum(self.grad*self.x.data)

In [14]:
# a class for an array that's the result of a division operation
class BA_Div(BackproppableArray):
    # x / y
    def __init__(self, x, y):
        super().__init__(x.data / y.data, [x,y])
        self.x = x
        self.y = y

    def grad_fn(self):
        # TODO: (1.3, 2.3) implement grad fn for Div
        # self.x.grad += (1.0 / self.y.data)*self.grad
        # self.y.grad -= (self.x.data* self.grad)/(self.y.data**2)
        
        if len(self.x.data.shape)==0:
            self.x.grad = self.x.grad+(1.0 / self.y.data)*self.grad
        else:
            if len(self.x.data) != 1:
                self.x.grad = self.x.grad+(1.0 / self.y.data)*self.grad
            else:
                self.x.grad = self.x.grad+np.sum(self.grad/self.y.data)
        if len(self.y.data.shape)==0:
            self.y.grad = self.y.grad-(self.x.data* self.grad)/(self.y.data**2)
        else:
            if len(self.y.data) != 1:
                self.y.grad = self.y.grad-(self.x.data* self.grad)/(self.y.data**2)
            else:
                self.y.grad = self.y.grad-np.sum(self.grad*self.x.data/self.y.data**2)

In [15]:
# a class for an array that's the result of a matrix multiplication operation
class BA_MatMul(BackproppableArray):
    # x @ y
    def __init__(self, x, y):
        # we only support multiplication of matrices, i.e. arrays with shape of length 2
        assert(len(x.data.shape) == 2)
        assert(len(y.data.shape) == 2)
        super().__init__(x.data @ y.data, [x,y])
        self.x = x
        self.y = y

    def grad_fn(self):
        # TODO: (2.1) implement grad fn for MatMul
        self.x.grad = self.x.grad+self.grad@self.y.data.T
        self.y.grad = self.y.grad+self.x.data.T@self.grad

In [16]:
# a class for an array that's the result of an exponential operation
class BA_Exp(BackproppableArray):
    # exp(x)
    def __init__(self, x):
        super().__init__(np.exp(x.data), [x])
        self.x = x

    def grad_fn(self):
        # TODO: (1.3) implement grad fn for Exp
        self.x.grad = self.x.grad+np.exp(self.x.data)*self.grad

In [17]:
def exp(x):
    if isinstance(x, BackproppableArray):
        return BA_Exp(x)
    else:
        return np.exp(x)

In [18]:
# a class for an array that's the result of an logarithm operation
class BA_Log(BackproppableArray):
    # log(x)
    def __init__(self, x):
        super().__init__(np.log(x.data), [x])
        self.x = x

    def grad_fn(self):
        # TODO: (1.3) implement grad fn for Log
        self.x.grad = self.x.grad+(1.0 / self.x.data)*self.grad

In [19]:
def log(x):
    if isinstance(x, BackproppableArray):
        return BA_Log(x)
    else:
        return np.log(x)

In [20]:
# a class for an array that's the result of a sum operation
class BA_Sum(BackproppableArray):
    # x.sum(axis, keepdims=True)
    def __init__(self, x, axis):
        super().__init__(x.data.sum(axis, keepdims=True), [x])
        self.x = x
        self.axis = axis

    def grad_fn(self):
        # TODO: (2.1) implement grad fn for Sum
        if len(self.x.data.shape)==0:
            self.x.grad = self.x.grad+self.grad
        else:
            self.x.grad = self.x.grad+self.grad*np.ones(self.x.data.shape)

In [21]:
# a class for an array that's the result of a reshape operation
class BA_Reshape(BackproppableArray):
    # x.reshape(shape)
    def __init__(self, x, shape):
        super().__init__(x.data.reshape(shape), [x])
        self.x = x
        self.shape = shape

    def grad_fn(self):
        # TODO: (2.1) implement grad fn for Reshape
        if type(self.grad)!=float:
               self.x.grad = self.x.grad+self.grad.reshape(self.x.data.shape)

In [22]:
# a class for an array that's the result of a transpose operation
class BA_Transpose(BackproppableArray):
    # x.transpose(axes)
    def __init__(self, x, axes):
        super().__init__(x.data.transpose(axes), [x])
        self.x = x
        self.axes = axes

    def grad_fn(self):
        # TODO: (2.1) implement grad fn for Transpose
        if type(self.grad)!=float:
            self.x.grad = self.x.grad+np.transpose(self.grad, axes = self.axes)

In [23]:
class BA_Sigmoid(BackproppableArray):
    def __init__(self, x):
        super().__init__(1 / (1 + np.exp(-x.data)), [x])
        self.x = x
    def grad_fn(self):
        self.x.grad = self.x.grad + 1 / (1 + np.exp(-self.x.data)) * (1 - 1 / (1 + np.exp(-self.x.data))) * self.grad

In [24]:
def sigmoid(x):
    if isinstance(x, BackproppableArray):
        return BA_Sigmoid(x)
    else:
        return 1 / (1 + np.exp(-x))

In [25]:
# automatic derivative of scalar function f at x, using backprop
def backprop_diff(f, x):
    ba_x = to_ba(x)
    fx = f(ba_x)
    fx.backward()
    return ba_x.grad

In [32]:
# SGD + Momentum (threaded) for Neural Network
#
# Xs              training examples (d * n)
# Ys              training labels   (c * n)
# gamma           L2 regularization constant
# W0              the initial value of the parameters (c * d)
# alpha           step size/learning rate
# beta            momentum hyperparameter
# B               minibatch size
# num_epochs      number of epochs (passes through the training set) to run
# monitor_period  how frequently, in terms of batches (not epochs) to output the parameter vector
# num_threads     how many threads to use
#
# returns         the final model arrived at at the end of training
def nn_sgd_mss_with_momentum_threaded(Xs, Ys, gamma, W10, W20, alpha, beta, B, num_epochs, num_threads):
    (d, n) = Xs.shape
    (h, d) = W10.shape
    # perform global setup/initialization/allocation
    V1 = np.zeros(W10.shape)
    V2 = np.zeros(W20.shape)
    W1 = np.copy(W10)
    W2 = np.copy(W20)
    gradient1 = np.zeros(W10.shape)
    gradient2 = np.zeros(W20.shape)
    Bt = int(B / num_threads)

    # construct the barrier object
    iter_barrier = threading.Barrier(num_threads + 1)

    # a function for each thread to run
    def thread_main(ithread):
        # perform any per-thread allocations
        # avoid memory allocation in the running of program by pre-allocating memories ahead
        thread_gradient1 = np.zeros(W10.shape)
        thread_gradient2 = np.zeros(W20.shape)

        slices_X = []
        slices_Y = []
        for ibatch in range(int(n/B)):
            ii = range(ibatch*B + ithread*Bt, ibatch*B + (ithread+1)*Bt)
            slices_X.append(np.ascontiguousarray(Xs[:,ii]))
            slices_Y.append(np.ascontiguousarray(Ys[:,ii]))
        # gradint calculation (uses only pre-allocated memories to improve performance of SGD)
        for it in range(num_epochs):
            for ibatch in range(int(n/B)):
                # work done by thread in each iteration;
                # this section of code primarily uses numpy operations with the "out=" argument specified
                np.multiply(thread_gradient1, 0, out=thread_gradient1)
                np.add(thread_gradient1,\
                       backprop_diff(lambda W: neural_network_loss_1(slices_X[ibatch], slices_Y[ibatch], W, W2), W1), out=thread_gradient1)
                np.multiply(thread_gradient2, 0, out=thread_gradient2)
                np.add(thread_gradient2,\
                       backprop_diff(lambda W: neural_network_loss_2(slices_X[ibatch], slices_Y[ibatch], W1, W), W2), out=thread_gradient2)
                
                iter_barrier.wait() # wait for all threads to finish computation before moving up to next step
                np.add(gradient1, thread_gradient1, out=gradient1)
                np.add(gradient2, thread_gradient2, out=gradient2)
                
                iter_barrier.wait()

    worker_threads = [threading.Thread(target=thread_main, args=(it,)) for it in range(num_threads)]

    for t in worker_threads:
        print("running thread ", t)
        t.start()

    print("Running minibatch sequential-scan SGD with momentum (%d threads)" % num_threads)
    # gradient & momentum update:
    # v <- beta * v - alpha * gradient
    # w <- w + v
    for it in tqdm(range(num_epochs)):
        for ibatch in range(int(n/B)):
            np.multiply(gradient1, 0, out=gradient1)
            np.multiply(gradient2, 0, out=gradient2)
            iter_barrier.wait()
            # work done on a single thread at each iteration;
            # this section of code primarily uses numpy operations with the "out=" argument specified
            np.divide(gradient1, B, out=gradient1)
            np.multiply(beta, V1, out=V1)
            np.multiply(alpha, gradient1, out=gradient1)
            np.subtract(V1, gradient1, out=V1)
            np.add(W1, V1, out=W1)
            np.divide(gradient2, B, out=gradient2)
            np.multiply(beta, V2, out=V2)
            np.multiply(alpha, gradient2, out=gradient2)
            np.subtract(V2, gradient2, out=V2)
            np.add(W2, V2, out=W2)
            iter_barrier.wait()

    for t in worker_threads:
        t.join()

    print("current loss: " + str((W2 @ sigmoid(W1 @ Xs) - Ys) @ (W2 @ sigmoid(W1 @ Xs) - Ys).transpose())) # report current loss
    # return the learned model
    return W1, W2

In [33]:
# customized hyperparameter tryout
nn_sgd_mss_with_momentum_threaded(Xs=Xs_tr, Ys=Ys_tr, gamma=0.0001, W10=W10, W20=W20, alpha=0.001, beta=0.9, B=8, num_epochs=200, num_threads=8)

running thread  <Thread(Thread-11, initial)>
running thread  <Thread(Thread-12, initial)>
running thread  <Thread(Thread-13, initial)>
running thread  <Thread(Thread-14, initial)>
running thread  <Thread(Thread-15, initial)>
running thread  <Thread(Thread-16, initial)>
running thread  <Thread(Thread-17, initial)>
running thread  <Thread(Thread-18, initial)>
Running minibatch sequential-scan SGD with momentum (8 threads)


100%|██████████| 200/200 [00:18<00:00, 10.98it/s]

current loss: [[49.55791879]]


(array([[-2.55844122e-03, -3.35173051e-03, -5.30038179e-03,
          2.25961259e-03,  5.54046327e-03,  7.06139460e-05,
         -1.27880822e-05,  6.14877023e-03, -7.04510279e-03,
         -5.11654413e-03, -4.61385844e-03,  9.98700775e-04,
          1.34734094e-03,  2.08066971e-05, -9.06537543e-05,
          4.36028589e-03, -4.43149891e-03, -1.09513450e-02,
         -3.75449972e-03,  4.73086053e-03,  5.15074252e-03,
          3.98984900e-03, -6.91025082e-03,  8.58127200e-03,
         -4.58574768e-03,  3.90778388e-03, -1.04859179e-03,
          6.61668308e-04,  8.97947829e-03, -4.67860270e-03],
        [-2.55844122e-03, -3.35173051e-03, -5.30038179e-03,
          2.25961259e-03,  5.54046327e-03,  7.06139460e-05,
         -1.27880822e-05,  6.14877023e-03, -7.04510279e-03,
         -5.11654413e-03, -4.61385844e-03,  9.98700775e-04,
          1.34734094e-03,  2.08066971e-05, -9.06537543e-05,
          4.36028589e-03, -4.43149891e-03, -1.09513450e-02,
         -3.75449972e-03,  4.73086053e-

In [34]:
# gradient descent to do the inner optimization step of Bayesian optimization
#
# objective     the objective function to minimize, as a function that takes a torch tensor and returns an expression
# x0            initial value to assign to variable (torch tensor)
# alpha         learning rate/step size
# num_iters     number of iterations of gradient descent
#
# returns     (obj_min, x_min), where
#       obj_min     the value of the objective after running iterations of gradient descent
#       x_min       the value of x after running iterations of gradient descent
def gradient_descent(objective, x0, alpha, num_iters):
    x = x0.detach().clone()  # create a fresh copy of x0
    x.requires_grad = True   # make it a target for differentiation
    opt = torch.optim.SGD([x], alpha)
    for it in range(num_iters):
        opt.zero_grad()
        f = objective(x)
        f.backward()
        opt.step()
    x.requires_grad = False  # make x no longer require gradients
    return (float(f.item()), x)

In [35]:
# compute the Gaussian RBF kernel matrix for a vector of data points (in PyTorch)
#
# Xs        points at which to compute the kernel (size: d x m)
# Zs        other points at which to compute the kernel (size: d x n)
# gamma     gamma parameter for the RBF kernel
#
# returns   an (m x n) matrix Sigma where Sigma[i,j] = K(Xs[:,i], Zs[:,j])
def rbf_kernel_matrix(Xs, Zs, gamma):
    m = Xs.shape[1] if len(Xs.shape) > 1 else 1
    n = Zs.shape[1] if len(Zs.shape) > 1 else 1
    sigma = [[torch.exp(-gamma * torch.linalg.norm(Xs[:, i] - Zs[:, j])**2) for j in range(n)] for i in range(m)]
    sigma = torch.tensor(sigma)
    return sigma

In [36]:
# compute the distribution predicted by a Gaussian process that uses an RBF kernel (in PyTorch)
#
# Xs            points at which to compute the kernel (size: d x n) where d is the number of parameters
# Ys            observed value at those points (size: n)
# gamma         gamma parameter for the RBF kernel
# sigma2_noise  the variance sigma^2 of the additive gaussian noise used in the model
#
# returns   a function that takes a value Xtest (size: d) and returns a tuple (mean, variance)
def gp_prediction(Xs, Ys, gamma, sigma2_noise):
    # first, do any work that can be shared among predictions
    sigma = rbf_kernel_matrix(Xs, Xs, gamma)
    n = Xs.shape[1]
    # next, define a nested function to return
    def prediction_mean_and_variance(Xtest):
        # construct mean and variance
        k = [torch.exp(-gamma * torch.linalg.norm(Xs[:, i] - Xtest)) for i in range(n)]
        k = torch.tensor(k)
        
        mean = k @ torch.linalg.inv(sigma + sigma2_noise * torch.eye(n)) @ Ys
        variance = torch.exp(-gamma * torch.linalg.norm(Xtest - Xtest)) + sigma2_noise -\
        k @ torch.linalg.inv(sigma + sigma2_noise * torch.eye(n)) @ k.T
        return (mean.reshape(()), variance.reshape(()))
    #finally, return the nested function
    return prediction_mean_and_variance

In [37]:
# run Bayesian optimization to minimize an objective
#
# objective     objective function; takes a torch tensor, returns a python float scalar
# d             dimension to optimize over
# gamma         gamma to use for RBF hyper-hyperparameter
# sigma2_noise  additive Gaussian noise parameter for Gaussian Process
# acquisition   acquisition function to use (e.g. ei_acquisition)
# random_x      function that returns a random sample of the parameter we're optimizing over (a torch tensor, e.g. for use in warmup)
# gd_nruns      number of random initializations we should use for gradient descent for the inner optimization step
# gd_alpha      learning rate for gradient descent
# gd_niters     number of iterations for gradient descent
# n_warmup      number of initial warmup evaluations of the objective to use
# num_iters     number of outer iterations of Bayes optimization to run (including warmup)
#
# returns       tuple of (y_best, x_best, Ys, Xs), where
#   y_best          objective value of best point found
#   x_best          best point found
#   Ys              vector of objective values for all points searched (size: num_iters)
#   Xs              matrix of all points searched (size: d x num_iters)
def bayes_opt(objective, d, gamma, sigma2_noise, acquisition, random_x, gd_nruns, gd_alpha, gd_niters, n_warmup, num_iters):
    y_best = float("inf")
    x_best = torch.zeros(size=(d,))
    Xs = []
    Ys = []
    # warm-up to prepare prior information for Bayesian Optimization
    for _ in range(n_warmup):
        x_i = torch.tensor([0.0001, 0.001, 0.9])
        y_i = objective(x_i)
        Xs.append(x_i)
        Ys.append(y_i)
        if y_i <= y_best:
            y_best = y_i
            x_best = x_i
    for _ in range(n_warmup, num_iters):
        Xs_vec = torch.stack(tensors=Xs, dim=1)
        Ys_vec = torch.tensor(Ys)
        prediction_fn =  gp_prediction(Xs_vec, Ys_vec, gamma, sigma2_noise)
        y = float("inf")
        x = torch.zeros(size=(d,))
        for _ in range(gd_nruns):
            x_0 = random_x(size=(d,))
            _, x_i = gradient_descent(objective=lambda x: acquisition(y_best, prediction_fn(x)[0], torch.sqrt(prediction_fn(x)[1])),\
                x0=x_0, alpha=gd_alpha, num_iters=gd_niters)
            y_i = objective(x_i)
            if y_i <= y:
                y = y_i
                x = x_i
        Xs.append(x)
        Ys.append(y)
        if y <= y_best:
            y_best = y
            x_best = x
    Xs_vec = torch.stack(tensors=Xs, dim=1)
    Ys_vec = torch.tensor(Ys)
    return y_best, x_best, Ys_vec, Xs_vec

In [38]:
# return a function that computes the lower confidence bound (LCB) acquisition function
#
# kappa     parameter for LCB
#
# returns   function that computes the LCB acquisition function
def lcb_acquisition(kappa):
    def A_lcb(Ybest, mean, stdev):
        return mean - kappa * stdev
    return A_lcb

In [41]:
# produce a function that runs SGD+Momentum on the receipts dataset, initializing the weights to zero
#
# mnist_dataset         the MNIST dataset, as returned by load_MNIST_dataset_with_validation_split
# num_epochs            number of epochs to run for
# B                     the batch size
#
# returns               a function that takes parameters
#   params                  a numpy vector of shape (3,) with entries that determine the hyperparameters, where
#       gamma = params[0]
#       alpha = params[1]
#       beta = params[2]
#                       and returns (the validation error of the final trained model after all the epochs) minus 0.9.
#                       if training diverged (i.e. any of the weights are non-finite) then return 0.1, which corresponds to an error of 1.
def receipts_dataset_sgd_mss_with_momentum(receipts_dataset, B, num_epochs, num_threads):
    def objective(params):
        Xs_tr, Xs_va, Ys_tr, Ys_va = receipts_dataset
        d = Xs_tr.shape[0]
        c = Ys_tr.shape[0]
        if torch.is_tensor(Xs_tr):
            Xs_tr = Xs_tr.numpy()
        if torch.is_tensor(Ys_tr):
            Ys_tr = Ys_tr.numpy()
        if torch.is_tensor(Xs_va):
            Xs_va = Xs_va.numpy()
        if torch.is_tensor(Ys_va):
            Ys_va = Ys_va.numpy()
        gamma, alpha, beta, W_0 = float(params[0].item()), float(params[1].item()), float(params[2].item()), np.zeros(shape=(c,d))
        W1, W2 = nn_sgd_mss_with_momentum_threaded(Xs=Xs_tr, Ys=Ys_tr, gamma=gamma, W10=W10, W20=W20, alpha=alpha, beta=beta,\
                                           B=B, num_epochs=num_epochs, num_threads=num_threads)
        Ys_pr = W2 @ sigmoid(W1 @ Xs_va)
        error = (W2 @ sigmoid(W1 @ Xs_va) - Ys_va) @ (W2 @ sigmoid(W1 @ Xs_va) - Ys_va).transpose() # use 1 - R^2 as error to select hyperparameters
        return float(error)
    return objective

In [42]:
# perform Bayesian Optimization to find optimal hyperparameters
obj = receipts_dataset_sgd_mss_with_momentum(receipts_dataset, B=8, num_epochs=40, num_threads=8)
(y_best, x_best, Ys_vec, Xs_vec) = bayes_opt(objective=obj, d=3, gamma=10, sigma2_noise=0.001, acquisition=lcb_acquisition(kappa=2.0),\
                                     random_x=torch.randn, gd_nruns=20, gd_alpha=0.01, gd_niters=20, n_warmup=3, num_iters=20)
print(y_best) # best R^2 score
print(x_best) # best hyperparameter set
print(Ys_vec) # R^2 score history
print(Xs_vec) # hyperparameter set history

running thread  <Thread(Thread-28, initial)>
running thread  <Thread(Thread-29, initial)>
running thread  <Thread(Thread-30, initial)>
running thread  <Thread(Thread-31, initial)>
running thread  <Thread(Thread-32, initial)>
running thread  <Thread(Thread-33, initial)>
running thread  <Thread(Thread-34, initial)>
running thread  <Thread(Thread-35, initial)>
Running minibatch sequential-scan SGD with momentum (8 threads)


100%|██████████| 40/40 [00:02<00:00, 17.35it/s]
<ipython-input-24-3118aad6faaf>:5: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


current loss: [[49.4804125]]
running thread  <Thread(Thread-36, initial)>
running thread  <Thread(Thread-37, initial)>
running thread  <Thread(Thread-38, initial)>
running thread  <Thread(Thread-39, initial)>
running thread  <Thread(Thread-40, initial)>
running thread  <Thread(Thread-41, initial)>
running thread  <Thread(Thread-42, initial)>
running thread  <Thread(Thread-43, initial)>
Running minibatch sequential-scan SGD with momentum (8 threads)


100%|██████████| 40/40 [00:02<00:00, 17.79it/s]


current loss: [[49.43518533]]
running thread  <Thread(Thread-44, initial)>
running thread  <Thread(Thread-45, initial)>
running thread  <Thread(Thread-46, initial)>
running thread  <Thread(Thread-47, initial)>
running thread  <Thread(Thread-48, initial)>
running thread  <Thread(Thread-49, initial)>
running thread  <Thread(Thread-50, initial)>
running thread  <Thread(Thread-51, initial)>
Running minibatch sequential-scan SGD with momentum (8 threads)


...

current loss: [[4.2311124e+145]]
running thread  <Thread(Thread-2756, initial)>
running thread  <Thread(Thread-2757, initial)>
running thread  <Thread(Thread-2758, initial)>
running thread  <Thread(Thread-2759, initial)>
running thread  <Thread(Thread-2760, initial)>
running thread  <Thread(Thread-2761, initial)>
running thread  <Thread(Thread-2762, initial)>
running thread  <Thread(Thread-2763, initial)>
Running minibatch sequential-scan SGD with momentum (8 threads)


100%|██████████| 40/40 [00:03<00:00, 12.40it/s]


current loss: [[2.75771984e+123]]
running thread  <Thread(Thread-2764, initial)>
running thread  <Thread(Thread-2765, initial)>
running thread  <Thread(Thread-2766, initial)>
running thread  <Thread(Thread-2767, initial)>
running thread  <Thread(Thread-2768, initial)>
running thread  <Thread(Thread-2769, initial)>
running thread  <Thread(Thread-2770, initial)>
running thread  <Thread(Thread-2771, initial)>
Running minibatch sequential-scan SGD with momentum (8 threads)


100%|██████████| 40/40 [00:02<00:00, 13.44it/s]

current loss: [[nan]]
5319271672313898.0
tensor([-1.0303,  0.3383,  1.0145])
tensor([5.3257e+15, 5.3257e+15, 5.3257e+15, 5.3257e+15, 5.3257e+15, 5.3257e+15,
        5.3257e+15, 5.3257e+15, 5.3257e+15, 5.3257e+15, 5.3257e+15, 5.3257e+15,
        5.3257e+15, 5.3257e+15, 5.3257e+15, 5.3257e+15, 5.3257e+15, 5.3193e+15,
        5.3257e+15, 5.3257e+15])
tensor([[ 1.0000e-04,  1.0000e-04,  1.0000e-04, -1.3602e+00,  6.2964e-01,
          8.4174e-01, -2.5236e-01,  6.0525e-01,  1.7894e+00,  6.1155e-02,
          8.5220e-01, -1.7842e+00, -7.3041e-01,  1.1160e+00,  3.7798e-01,
         -2.0286e+00, -5.3640e-01, -1.0303e+00, -5.8221e-02, -4.2254e-01],
        [ 1.0000e-03,  1.0000e-03,  1.0000e-03,  6.7530e-01,  5.9039e-01,
          5.6459e-01,  1.2596e+00,  6.7088e-01,  1.4162e+00,  9.2197e-01,
          3.4475e-01,  7.6778e-01,  1.6476e+00,  2.6488e+00,  1.0802e+00,
          1.9088e+00,  9.4263e-01,  3.3829e-01,  5.0835e-01,  2.4066e-01],
        [ 9.0000e-01,  9.0000e-01,  9.0000e-01,  1.3761e

In [43]:
# customized hyperparameter tryout
W1, W2 =\
nn_sgd_mss_with_momentum_threaded(Xs=Xs_tr, Ys=Ys_tr, gamma=0.0001, W10=W10, W20=W20, alpha=0.001, beta=0.9, B=8, num_epochs=200, num_threads=8)

running thread  <Thread(Thread-2772, initial)>
running thread  <Thread(Thread-2773, initial)>
running thread  <Thread(Thread-2774, initial)>
running thread  <Thread(Thread-2775, initial)>
running thread  <Thread(Thread-2776, initial)>
running thread  <Thread(Thread-2777, initial)>
running thread  <Thread(Thread-2778, initial)>
running thread  <Thread(Thread-2779, initial)>
Running minibatch sequential-scan SGD with momentum (8 threads)


100%|██████████| 200/200 [00:12<00:00, 16.16it/s]

current loss: [[49.42656076]]


In [48]:
W1 @ np.array([Receipt_Count[j] for j in range(364, 364 - day_range, -1)], dtype=float).reshape(30,)

array([-90983.71200771, -90983.71200771, -90983.71200771, -90983.71200771,
       -90983.71200771])

In [49]:
# use the model to predict the approximate number of the scanned receipts for each day of 2022
q = deque()
warm_up = [Receipt_Count[j] for j in range(364, 364 - day_range, -1)]
for count in warm_up:
    q.append(count)
predictions = []
for _ in range(365):
    X = np.array(q, dtype=float).reshape(30,)
    predictions.append(((W2 @ sigmoid(W1 @ (X - mean) / std)) * std + mean).item())
    q.popleft()
    q.append(predictions[-1])
# calculate the predicted number of the scanned receipts for each month of 2022
df = pandas.DataFrame()
df.index = pandas.date_range(start='1/1/2022', end='12/31/2022')
df["count"] = predictions
monthSum = df.groupby(df.index.month).sum()

In [50]:
df # predicted data

,count
2022-01-01,8.871687e+06
2022-01-02,8.873233e+06
2022-01-03,8.872166e+06
2022-01-04,8.871940e+06
2022-01-05,8.872501e+06
...,...
2022-12-27,8.872248e+06
2022-12-28,8.872248e+06
2022-12-29,8.872248e+06
2022-12-30,8.872248e+06
